In [1]:
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import warnings
warnings.filterwarnings('ignore')

# Kiểm tra GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Cài đặt seed để đảm bảo kết quả nhất quán
torch.manual_seed(42)
np.random.seed(42)

d:\Kaggle\.ssh\news_project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu


In [2]:
%pip install transformers==4.40.0


Note: you may need to restart the kernel to use updated packages.


# Đọc và xử lý dữ liệu

In [3]:
print("Loading data...")
df = pd.read_csv('clean_news.csv')

print(f"Original data shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

# Bỏ các dòng thiếu dữ liệu
df = df.dropna(subset=["clean_text", "label"])
print(f"After dropping NaN: {df.shape}")

# Ép kiểu và loại bỏ dòng rỗng
df["clean_text"] = df["clean_text"].astype(str).str.strip()
df = df[df["clean_text"] != ""]
print(f"After removing empty text: {df.shape}")

# Ép kiểu nhãn
df["label"] = df["label"].astype(int)

# Hiển thị thống kê
print(f"Label distribution:\n{df['label'].value_counts()}")
print(f"Sample texts:")
for i in range(min(3, len(df))):
    print(f"Label {df.iloc[i]['label']}: {df.iloc[i]['clean_text'][:100]}...")

Loading data...
Original data shape: (8697, 10)
Columns: ['text', 'label', 'id', 'user_name', 'post_message', 'timestamp_post', 'num_like_post', 'num_comment_post', 'num_share_post', 'clean_text']
After dropping NaN: (4225, 10)
After removing empty text: (4225, 10)
Label distribution:
label
0    4225
Name: count, dtype: int64
Sample texts:
Label 0: Bác_sĩ TP HCM thắng giải nhiếp_ảnh thế_giới Đầu tháng 7 bác_sĩ Hoài_Anh háo_hức chờ đón chiếc cúp Nh...
Label 0: 11 cán_bộ phải quay lại làm_việc sau nửa năm nghỉ hưu Ngày 276 lãnh_đạo huyện Quỳnh_Lưu cho biết đã ...
Label 0: Thêm hơn 210000 người Nga đăng_ký tham_chiến ở Ukraine_Đây là thành_quả phối_hợp của tất_cả các cơ_q...


# Tách dữ liệu train/validation

In [4]:
texts = df["clean_text"].tolist()
labels = df["label"].tolist()

print(f"Total samples: {len(texts)}")
print(f"Unique labels: {set(labels)}")

train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

print(f"Train samples: {len(train_texts)}")
print(f"Validation samples: {len(val_texts)}")
print(f"Train label distribution: {pd.Series(train_labels).value_counts().to_dict()}")
print(f"Validation label distribution: {pd.Series(val_labels).value_counts().to_dict()}")

Total samples: 4225
Unique labels: {0}
Train samples: 3380
Validation samples: 845
Train label distribution: {0: 3380}
Validation label distribution: {0: 845}


# Tokenization với PhoBERT

In [5]:
print("Loading PhoBERT tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

print("Tokenizing training data...")
train_encodings = tokenizer(
    train_texts, 
    padding=True, 
    truncation=True, 
    max_length=256, 
    return_tensors="pt"
)

print("Tokenizing validation data...")
val_encodings = tokenizer(
    val_texts, 
    padding=True, 
    truncation=True, 
    max_length=256, 
    return_tensors="pt"
)

print(f"Training encodings shape: {train_encodings['input_ids'].shape}")
print(f"Validation encodings shape: {val_encodings['input_ids'].shape}")

Loading PhoBERT tokenizer...
Tokenizing training data...
Tokenizing validation data...
Training encodings shape: torch.Size([3380, 256])
Validation encodings shape: torch.Size([845, 256])


# Tạo Dataset

In [6]:

class NewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        item["labels"] = label
        return item

train_dataset = NewsDataset(train_encodings, train_labels)
val_dataset = NewsDataset(val_encodings, val_labels)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

# Test một sample
sample = train_dataset[0]
print(f"Sample keys: {sample.keys()}")
print(f"Sample input_ids shape: {sample['input_ids'].shape}")
print(f"Sample label: {sample['labels']}")
print(f"Sample label dtype: {sample['labels'].dtype}")
print(f"Sample label shape: {sample['labels'].shape}")

Train dataset size: 3380
Validation dataset size: 845
Sample keys: dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
Sample input_ids shape: torch.Size([256])
Sample label: 0
Sample label dtype: torch.int64
Sample label shape: torch.Size([])


# Tạo và cấu hình mô hình

In [7]:
num_labels = len(set(labels))
print(f"Number of labels: {num_labels}")
print(f"Unique labels: {sorted(set(labels))}")

print("Loading PhoBERT model...")
model = AutoModelForSequenceClassification.from_pretrained(
    "vinai/phobert-base", 
    num_labels=num_labels,
    problem_type="single_label_classification"
)


print(f"Model loaded successfully!")
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

print("Testing forward pass...")
test_batch = next(iter(train_dataset))
test_batch = {k: v.unsqueeze(0).to(device) for k, v in test_batch.items()}
with torch.no_grad():
    outputs = model(**test_batch)
    print(f"Output shape: {outputs.logits.shape}")
    print(f"Expected shape: [1, {num_labels}]")

Number of labels: 1
Unique labels: [0]
Loading PhoBERT model...


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded successfully!
Model parameters: 134,999,041
Testing forward pass...
Output shape: torch.Size([1, 1])
Expected shape: [1, 1]


# Cấu hình training arguments

In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,  
    per_device_eval_batch_size=4,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_strategy="steps",
    evaluation_strategy="steps",
    save_steps=500,
    eval_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to=None,
    dataloader_pin_memory=False,
    remove_unused_columns=False,
    fp16=False,
    dataloader_num_workers=0,
    gradient_accumulation_steps=4,
    learning_rate=1e-5,
)

print("✅ TrainingArguments đã được tạo thành công!")


✅ TrainingArguments đã được tạo thành công!


In [9]:

import transformers
print(transformers.__version__)


4.40.0


In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer
import torch

# Load lại tokenizer và model từ thư mục đã lưu
model_dir = "D:/Kaggle/.ssh/news_project/saved_model/"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSequenceClassification.from_pretrained(model_dir)

# Thiết bị huấn luyện (GPU nếu có, không thì CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Nếu bạn muốn tiếp tục train, thì cấu hình training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_strategy="steps",
    evaluation_strategy="steps",
    save_steps=500,
    eval_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to=None,
    dataloader_pin_memory=False,
    remove_unused_columns=False,
    fp16=False,
    dataloader_num_workers=0,
    gradient_accumulation_steps=4,
    learning_rate=1e-5,
)

# Nếu chưa tạo eval_dataset thì tạo tại đây
eval_dataset = NewsDataset(val_encodings, val_labels)

# Tạo Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

# Huấn luyện tiếp tục nếu muốn
print("=== Tiếp tục huấn luyện bằng Trainer ===")
try:
    trainer.train()

except KeyboardInterrupt:
    print("⛔ Đã dừng giữa chừng. Lưu model tạm thời...")
    model.save_pretrained("saved_model_partial/")
    print("✅ Huấn luyện đã được lưu tạm thời!")

except Exception as e:
    print(f"❌ Lỗi Trainer: {e}")
    print("⏳ Đang chuyển sang huấn luyện thủ công...")

    from torch.utils.data import DataLoader
    from torch.optim import AdamW
    from tqdm import tqdm

    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    val_loader = DataLoader(eval_dataset, batch_size=8, shuffle=False)

    optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)

    for epoch in range(3):
        model.train()
        total_loss = 0

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/3"):
            batch = {k: v.to(device) for k, v in batch.items()}
            batch["labels"] = batch["labels"].long()

            outputs = model(**batch)
            loss = outputs.loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"🔁 Epoch {epoch+1}: Avg Loss = {avg_loss:.4f}")

# ✅ Lưu mô hình cuối cùng
model.save_pretrained("saved_model/")
tokenizer.save_pretrained("saved_model/")
print("💾 Mô hình đã được lưu tại 'saved_model/'")


In [10]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Load lại từ thư mục đã lưu
model = AutoModelForSequenceClassification.from_pretrained("saved_model/")
tokenizer = AutoTokenizer.from_pretrained("saved_model/")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_path = "./saved_model"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load model (chú ý num_labels = 2 nếu là bài toán phân loại nhị phân)
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)


In [ ]:
for batch in train_dataloader:
    input_ids = batch['input_ids']
    print("Max input_id:", input_ids.max())
    print("Vocab size:", tokenizer.vocab_size)
    break


# Tạo Trainer và huấn luyện

In [12]:
# Cell 8: Training với progress tracking cho CPU
import time
from tqdm import tqdm
import os
import torch

print("Starting CPU-optimized training...")
print(f"Training on {len(train_dataset)} samples")
print(f"Validation on {len(val_dataset)} samples")

# Ước tính thời gian
estimated_time_per_epoch = len(train_dataset) / 4 * 0.1  # Giả sử 0.1s per batch
total_estimated_time = estimated_time_per_epoch * 3 / 60
print(f"Estimated training time: {total_estimated_time:.1f} minutes")

# Training với Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

start_time = time.time()
print("Starting training...")

try:
    trainer.train()
    training_time = (time.time() - start_time) / 60
    print(f"Training completed in {training_time:.1f} minutes!")
    
    # ✅ Lưu mô hình sau khi train thành công
    trainer.save_model("models/final_model")
    tokenizer.save_pretrained("models/final_model")
    print("Model saved to models/final_model")

except Exception as e:
    print(f"Training error: {e}")
    print("Falling back to manual training...")

    # Manual training fallback
    from torch.utils.data import DataLoader
    from torch.optim import AdamW

    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

    optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)

    for epoch in range(3):
        model.train()
        total_loss = 0

        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/3")

        for batch_idx, batch in enumerate(progress_bar):
            batch = {k: v.to(device) for k, v in batch.items()}
            batch['labels'] = batch['labels'].long()

            outputs = model(**batch)
            loss = outputs.loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # Update progress bar
            progress_bar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'Avg Loss': f'{total_loss/(batch_idx+1):.4f}'
            })

        print(f"Epoch {epoch+1}: Loss = {total_loss/len(train_loader):.4f}")
    
    # ✅ Lưu mô hình sau huấn luyện thủ công
    model_save_path = "models/manual_trained_model"
    os.makedirs(model_save_path, exist_ok=True)
    torch.save(model.state_dict(), f"{model_save_path}/pytorch_model.bin")
    tokenizer.save_pretrained(model_save_path)
    print(f"Manual training model saved to {model_save_path}")


Starting CPU-optimized training...
Training on 3380 samples
Validation on 845 samples
Estimated training time: 4.2 minutes
Starting training...


  8%|▊         | 50/633 [12:59<2:36:56, 16.15s/it]

{'loss': 0.415, 'grad_norm': 1.2937830686569214, 'learning_rate': 5e-06, 'epoch': 0.24}


 16%|█▌        | 100/633 [24:13<1:55:28, 13.00s/it]

{'loss': 0.0285, 'grad_norm': 0.13045376539230347, 'learning_rate': 1e-05, 'epoch': 0.47}


...

{'loss': 0.0012, 'grad_norm': 0.018626516684889793, 'learning_rate': 1.557223264540338e-06, 'epoch': 2.6}


 95%|█████████▍| 600/633 [2:10:52<06:42, 12.19s/it]

{'loss': 0.0012, 'grad_norm': 0.018596813082695007, 'learning_rate': 6.191369606003752e-07, 'epoch': 2.84}


100%|██████████| 633/633 [2:17:37<00:00, 13.05s/it]

{'train_runtime': 8257.6918, 'train_samples_per_second': 1.228, 'train_steps_per_second': 0.077, 'train_loss': 0.036960163028648865, 'epoch': 3.0}
Training completed in 137.6 minutes!


In [13]:

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
print("Vocab size:", tokenizer.vocab_size)

# Kiểm tra xem có token nào vượt quá vocab không
for batch in train_dataset:
    if max(batch["input_ids"]) >= tokenizer.vocab_size:
        print("Found invalid token id:", max(batch["input_ids"]))


Vocab size: 64000


In [ ]:
for batch in train_dataset:
    input_ids = batch["input_ids"]
    if input_ids.max() >= model.config.vocab_size:
        print("Có token vượt quá vocab_size!")
        break


# Đánh giá mô hình

In [ ]:
print(type(trainer))  # Kết quả phải là <class 'transformers.trainer.Trainer'>


In [14]:
print("Evaluating model...")

# Đánh giá
eval_results = trainer.evaluate()
print(eval_results)


# Dự đoán
print("Making predictions...")
predictions = trainer.predict(val_dataset)
pred_labels = np.argmax(predictions.predictions, axis=1)

# Tính metrics
accuracy = accuracy_score(val_labels, pred_labels)
print(f"Accuracy: {accuracy:.4f}")

# Classification report
print("\nClassification Report:")
print(classification_report(val_labels, pred_labels))

# Confusion matrix
print("\nConfusion Matrix:")
cm = confusion_matrix(val_labels, pred_labels)
print(cm)

# Lưu model
print("\nSaving model...")
trainer.save_model("./best_model")
tokenizer.save_pretrained("./best_model")
print("Model saved to ./best_model/")

Evaluating model...


100%|██████████| 212/212 [02:40<00:00,  1.32it/s]


{'eval_loss': 0.0008740455377846956, 'eval_runtime': 161.6071, 'eval_samples_per_second': 5.229, 'eval_steps_per_second': 1.312, 'epoch': 2.996449704142012}
Making predictions...


100%|██████████| 212/212 [02:38<00:00,  1.33it/s]


Accuracy: 1.0000

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       845

    accuracy                           1.00       845
   macro avg       1.00      1.00      1.00       845
weighted avg       1.00      1.00      1.00       845


Confusion Matrix:
[[845]]

Saving model...
Model saved to ./best_model/


In [15]:

print("Saving model...")
trainer.save_model("./best_model")
tokenizer.save_pretrained("./best_model")
print("Model saved to ./best_model/")

# Test loading model
print("Testing model loading...")
loaded_model = AutoModelForSequenceClassification.from_pretrained("./best_model")
loaded_tokenizer = AutoTokenizer.from_pretrained("./best_model")
print("Model loaded successfully!")

Saving model...
Model saved to ./best_model/
Testing model loading...
Model loaded successfully!


In [16]:
# Cell 10: Test model đã train
print("Testing saved model...")

# Load model đã train
loaded_model = AutoModelForSequenceClassification.from_pretrained("./best_model")
loaded_tokenizer = AutoTokenizer.from_pretrained("./best_model")

# Test với một sample
if len(val_texts) > 0:
    test_text = val_texts[0]
    print(f"Test text: {test_text[:100]}...")
    
    # Tokenize
    inputs = loaded_tokenizer(
        test_text, 
        padding=True, 
        truncation=True, 
        max_length=256, 
        return_tensors="pt"
    )
    
    # Predict
    with torch.no_grad():
        outputs = loaded_model(**inputs)
        predictions = torch.softmax(outputs.logits, dim=-1)
        predicted_label = torch.argmax(predictions, dim=-1).item()
        confidence = predictions[0][predicted_label].item()
    
    print(f"Predicted label: {predicted_label}")
    print(f"Confidence: {confidence:.4f}")
    print(f"True label: {val_labels[0]}")
    print(f"Prediction correct: {predicted_label == val_labels[0]}")

Testing saved model...
Test text: Al Nassr lần thứ tư thay HLV từ khi có Ronaldo_Al Nassr thông_báo Pioli và đội_ngũ trợ_lý của ông sẽ...
Predicted label: 0
Confidence: 0.9992
True label: 0
Prediction correct: True


In [17]:
def predict_news(text, model, tokenizer, device):
    """
    Dự đoán nhãn cho một đoạn text
    """
    model.eval()
    
    # Tokenize
    inputs = tokenizer(
        text, 
        padding=True, 
        truncation=True, 
        max_length=256, 
        return_tensors="pt"
    )
    
    # Chuyển lên device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Dự đoán
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.softmax(outputs.logits, dim=-1)
        predicted_label = torch.argmax(predictions, dim=-1).item()
        confidence = predictions[0][predicted_label].item()
    
    return predicted_label, confidence

# Test hàm dự đoán
test_texts = [
    "Tin tức mới nhất về tình hình kinh tế Việt Nam",
    "Cập nhật về dịch bệnh COVID-19 tại Hà Nội",
    "Kết quả trận đấu bóng đá tối qua"
]

print("Testing prediction function:")
for i, text in enumerate(test_texts):
    predicted_label, confidence = predict_news(text, loaded_model, loaded_tokenizer, device)
    print(f"Text {i+1}: {text[:50]}...")
    print(f"  Predicted: {predicted_label}, Confidence: {confidence:.4f}")
    print()

Testing prediction function:
Text 1: Tin tức mới nhất về tình hình kinh tế Việt Nam...
  Predicted: 0, Confidence: 0.9982

Text 2: Cập nhật về dịch bệnh COVID-19 tại Hà Nội...
  Predicted: 0, Confidence: 0.9986

Text 3: Kết quả trận đấu bóng đá tối qua...
  Predicted: 0, Confidence: 0.9983

